# **MNIST classification with TensorFlow model**

In [25]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import os, sys

workdir_path = '/content/drive/My Drive/Python/ICA'
os.chdir(workdir_path)
sys.path.append(workdir_path)

Predictions 

In [27]:
import tensorflow as tf
tf.reset_default_graph()
# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data

import matplotlib.pyplot as plt     # only to show the images at end
from numpy.random import randint    # only to generate a random nuymber ar end
from numpy import argmax


from tensorflow.python.tools import freeze_graph
from tensorflow.python.tools import optimize_for_inference_lib

# load the data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from t

In [28]:
mnist


Datasets(train=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7f9211f73cc0>, validation=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7f9211f73e80>, test=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7f9211f73f28>)

In [29]:
sess = tf.Session()
graph = tf.get_default_graph()
with graph.as_default():
    with sess.as_default():
        #restoring the model
        saver = tf.train.import_meta_graph('tensorflowModel.ckpt.meta')
        saver.restore(sess,tf.train.latest_checkpoint('./'))

INFO:tensorflow:Restoring parameters from ./tensorflowModel.ckpt


In [30]:
import tensorflow as tf
gf = tf.GraphDef()
gf.ParseFromString(open('frozentensorflowModel.pb','rb').read())
[n.name + '=>' +  n.op for n in gf.node if n.op in ( 'Softmax','Placeholder')]

['x_1=>Placeholder', 'Placeholder=>Placeholder', 'Softmax=>Softmax']

In [14]:
sess = tf.Session()
graph = tf.get_default_graph()
with graph.as_default():
    with sess.as_default():
        #restoring the model
        saver = tf.train.import_meta_graph('tensorflowModel.ckpt.meta')
        saver.restore(sess,tf.train.latest_checkpoint('./'))
        #initializing all variables
        sess.run(tf.global_variables_initializer())
       
        #using the model for prediction
        x_tensor = graph.get_tensor_by_name("x_1:0")
        #keep_prob is not always necessary it depends on your model
        ##keep_prob = graph.get_tensor_by_name("dropout_keep_prob:0")
        op_to_restore = graph.get_tensor_by_name("Softmax:0")
        feed_dict={x_tensor: x_test,  keep_prob: 1.0}
        
        opt = []
        
        opt = sess.run(op_to_restore ,feed_dict)
        
        y_test = np.asarray(y_test)# converting list to numpy array
        labels=np.argmax(y_test, 1)
        predictions=np.argmax(opt,1)
        count =  np.sum(labels == predictions)
        print("Number of correct prediction %d out of %d"%(count,len(predictions)))
        print("Accuracy is {:.3f}". format(float(count)/len(predictions)))

INFO:tensorflow:Restoring parameters from ./tensorflowModel.ckpt


NameError: ignored

In [8]:
sess = tf.Session()
graph = tf.get_default_graph()
# with graph.as_default():
with sess.as_default():
    #restoring the model
    saver = tf.train.import_meta_graph('tensorflowModel.ckpt.meta')
    saver.restore(sess,tf.train.latest_checkpoint('./'))
    #initializing all variables
    sess.run(tf.global_variables_initializer())
    
    #using the model for prediction
    x_tensor = graph.get_tensor_by_name("x_1:0")
    #keep_prob is not always necessary it depends on your model
    ##keep_prob = graph.get_tensor_by_name("dropout_keep_prob:0")
    op_to_restore = graph.get_tensor_by_name("Softmax:0")
    feed_dict={x_tensor: X}
    
    opt = []
    
    opt = sess.run(op_to_restore ,feed_dict)
    
    Y = np.asarray(y_test)# converting list to numpy array
    labels=np.argmax(y_test, 1)
    predictions=np.argmax(opt,1)
    count =  np.sum(labels == predictions)
    print("Number of correct prediction %d out of %d"%(count,len(predictions)))
    print("Accuracy is {:.3f}". format(float(count)/len(predictions)))

INFO:tensorflow:Restoring parameters from ./tensorflowModel.ckpt


NameError: ignored

In [0]:
import tensorflow as tf

def load_graph(frozen_graph_filename):
    # We load the protobuf file from the disk and parse it to retrieve the 
    # unserialized graph_def
    with tf.gfile.GFile(frozen_graph_filename, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())

    # Then, we import the graph_def into a new Graph and returns it 
    with tf.Graph().as_default() as graph:
        # The name var will prefix every op/nodes in your graph
        # Since we load everything in a new graph, this is not needed
        tf.import_graph_def(graph_def, name="prefix")
    return graph

In [34]:

import argparse 
import tensorflow as tf

if __name__ == '__main__':


    # We use our "load_graph" function
    graph = load_graph('frozentensorflowModel.pb')

    with graph.as_default():
      # We can verify that we can access the list of operations in the graph
      for op in graph.get_operations():
          print(op.name)
          # prefix/Placeholder/inputs_placeholder
          # ...
          # prefix/Accuracy/predictions
          
      # We access the input and output nodes 
      input_node = graph.get_tensor_by_name('prefix/x_1:0')
      output_node = graph.get_tensor_by_name('prefix/Softmax:0')
          
      # We launch a Session
      with tf.Session(graph=graph) as sess:
          # Note: we don't nee to initialize/restore anything
          # There is no Variables in this graph, only hardcoded constants 
          print("")
          print("*********************************************")
          print("")
  #        y_out = sess.run(y, feed_dict={
  #            x: mnist.test.images[:256] # < 45
  #        })
          #sess.run(accuracy, feed_dict={X: mnist.test.images[:256], Y: mnist.test.labels[:256], keep_prob: 1.0})
          # I taught a neural net to recognise when a sum of numbers is bigger than 45
          # it should return False in this case
  #        print(y_out) # [[ False ]] Yay, it works!

          j=randint(1, mnist.test.images.shape[0])
          print(j)
          #plt.imshow(mnist.test.images[j].reshape((28,28)), cmap='Greys')    #  cmap='Greys' to show as greyscale
          print("prediction: ", argmax(sess.run(y, feed_dict={x: mnist.test.images[j:j+1]})))
          print("real value: ", argmax(mnist.test.labels[j:j+1]))

prefix/x_1
prefix/Placeholder
prefix/Variable
prefix/Variable/read
prefix/Variable_1
prefix/Variable_1/read
prefix/Variable_2
prefix/Variable_2/read
prefix/Variable_3
prefix/Variable_3/read
prefix/Variable_4
prefix/Variable_4/read
prefix/Variable_5
prefix/Variable_5/read
prefix/Variable_6
prefix/Variable_6/read
prefix/Variable_7
prefix/Variable_7/read
prefix/Reshape/shape
prefix/Reshape
prefix/Conv2D
prefix/BiasAdd
prefix/Relu
prefix/MaxPool
prefix/Conv2D_1
prefix/BiasAdd_1
prefix/Relu_1
prefix/MaxPool_1
prefix/Reshape_1/shape
prefix/Reshape_1
prefix/MatMul
prefix/Add
prefix/Relu_2
prefix/sub/x
prefix/sub
prefix/dropout/Shape
prefix/dropout/random_uniform/min
prefix/dropout/random_uniform/max
prefix/dropout/random_uniform/RandomUniform
prefix/dropout/random_uniform/sub
prefix/dropout/random_uniform/mul
prefix/dropout/random_uniform
prefix/dropout/sub/x
prefix/dropout/sub
prefix/dropout/truediv/x
prefix/dropout/truediv
prefix/dropout/GreaterEqual
prefix/dropout/mul
prefix/dropout/Cast
p

TypeError: ignored

In [22]:
input_node

<tf.Tensor 'prefix/x_1:0' shape=(?, 784) dtype=float32>

In [23]:
output_node

<tf.Tensor 'prefix/Softmax:0' shape=(?, 10) dtype=float32>

In [36]:
graph.shape


AttributeError: ignored